In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "../Qwen2.5-1.5B-Instruct-GRPO"

model_kwargs = {
    "torch_dtype": torch.bfloat16,
}

model = AutoModelForCausalLM.from_pretrained(model_path, **model_kwargs)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [5]:
from logging import getLogger as get_logger

log = get_logger(__name__)


# merge lora adapters and save
def merge_adapters_and_publish(
    model_id: str,
    torch_dtype=torch.float16,
    device_map={"": 0},
    merged_model_id: str=None,
):
    from peft import AutoPeftModelForCausalLM
    
    if isinstance(torch_dtype, str) and torch_dtype != "auto":
        torch_dtype = getattr(torch, torch_dtype) 

    log.info(f"Loading model and tokenizer for {model_id}")
    model = AutoPeftModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch_dtype,
        device_map=device_map,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

    log.info("Merging adapters to model...")
    model = model.merge_and_unload()

    if merged_model_id is None:
        merged_model_id = f"{model_id}-merged"
    log.info(f"Pushing merged model to HF hub as {merged_model_id}")
    model.push_to_hub(merged_model_id)
    tokenizer.push_to_hub(merged_model_id)
    return merged_model_id


In [6]:
merge_adapters_and_publish('../Qwen2.5-1.5B-Instruct-GRPO', merged_model_id='Qwen2.5-1.5B-Instruct-GRPO-merged')

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

'Qwen2.5-1.5B-Instruct-GRPO-merged'